In [ ]:
# =======================================================================================================
# PROBLEM C3
#
# Build a CNN based classifier for Cats vs Dogs dataset.
# Your input layer should accept 150x150 with 3 bytes color as the input shape.
# This is unlabeled data, use ImageDataGenerator to automatically label it.
# Don't use lambda layers in your model.
#
# The dataset used in this problem is originally published in https://www.kaggle.com/c/dogs-vs-cats/data
#
# Desired accuracy and validation_accuracy > 72%
# ========================================================================================================

In [30]:
import tensorflow as tf
import urllib.request
import zipfile
import tensorflow as tf
import os
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
def solution_C3():
    data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/cats_and_dogs.zip'
    urllib.request.urlretrieve(data_url, 'cats_and_dogs.zip')
    local_file = 'cats_and_dogs.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/')
    zip_ref.close()

    BASE_DIR = 'data/cats_and_dogs_filtered'
    train_dir = os.path.join(BASE_DIR, 'train')
    validation_dir = os.path.join(BASE_DIR, 'validation')

    train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
    
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(directory="data/cats_and_dogs_filtered/train/",
                                                      batch_size=100,
                                                      class_mode='binary',
                                                      target_size=(150, 150))
    
    validation_generator = test_datagen.flow_from_directory(directory="data/cats_and_dogs_filtered/validation/",
                                                                batch_size=100,
                                                                class_mode='binary',
                                                                target_size=(150, 150))

    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1024, activation='relu'),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1, activation='sigmoid')
      ])
  
    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        '''
        Halts the training after reaching 60 percent accuracy

        Args:
          epoch (integer) - index of epoch (required but unused in the function definition below)
          logs (dict) - metric results from the training epoch
        '''

        # Check accuracy
        if(logs.get('val_accuracy') > 0.75):

          # Stop if threshold is met
          print("\nLoss is lower than 0.4 so cancelling training!")
          self.model.stop_training = True

    callbacks = myCallback()

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=5e-4,                                                         
        decay_steps=200,
        decay_rate=0.90
    )

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=lr_schedule
    )

    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    
    model.fit(train_generator,
              epochs=50,
              steps_per_epoch=20,
              verbose=1,
              validation_data=validation_generator,
              validation_steps=10,
              callbacks=[callbacks])

    return model

In [42]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model = solution_C3()
    model.save("model_C3.h5")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/50
20/20 [==============================] - 57s 3s/step - loss: 1.0969 - accuracy: 0.5060 - val_loss: 0.6908 - val_accuracy: 0.5300
Epoch 2/50
20/20 [==============================] - 52s 3s/step - loss: 0.7017 - accuracy: 0.4875 - val_loss: 0.6866 - val_accuracy: 0.5750
Epoch 3/50
20/20 [==============================] - 52s 3s/step - loss: 0.6879 - accuracy: 0.5300 - val_loss: 0.6779 - val_accuracy: 0.6240
Epoch 4/50
20/20 [==============================] - 51s 3s/step - loss: 0.6812 - accuracy: 0.5795 - val_loss: 0.6623 - val_accuracy: 0.5870
Epoch 5/50
20/20 [==============================] - 51s 3s/step - loss: 0.6670 - accuracy: 0.5865 - val_loss: 0.6368 - val_accuracy: 0.6200
Epoch 6/50
20/20 [==============================] - 51s 3s/step - loss: 0.6517 - accuracy: 0.6110 - val_loss: 0.6282 - val_accuracy: 0.6150
Epoch 7/50
20/20 [==============================] - 52s 3s/step - loss: 0.64